In [1]:
!pip install transformers==4.37.2 optimum==1.12.0 --quiet
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --quiet
!pip install langchain==0.1.9 --quiet
# !pip install chromadb
!pip install sentence_transformers==2.4.0 --quiet
!pip install unstructured --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six==20221105 --quiet
!pip install unstructured-inference --quiet
!pip install faiss-gpu==1.7.2 --quiet
!pip install pikepdf==8.13.0 --quiet
!pip install pypdf==4.0.2 --quiet
!pip install pillow_heif==0.15.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [2]:
import json

# Opening JSON file
f_train = open('train_TLQA.json')
f_val = open('val_TLQA.json')
f_test = open('test_TLQA.json')

# returns JSON object as a dictionary
train_data = json.load(f_train)
val_data = json.load(f_val)
test_data = json.load(f_test)

In [3]:
random_question = "List all chairpersons of Ministry of Employment and Labor, also known as MOEL, from 2010 to 2020"
random_ground_truth = ["Yim Tae-hee (2010)", "Kim Young-joo (2017, 2018, 2019, 2020)"]

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class KnnSearch:
    def __init__(self,data=None, num_trees=None,emb_dim=None):
        self.num_trees=num_trees
        self.emb_dim=emb_dim
        self.model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

    def get_embeddings_for_data(self, data_ls):
        #model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        embeddings = self.model.encode(data_ls)
        return embeddings

    def get_top_n_neighbours(self, sentence, data_emb, transfer_data, k): #strategy_emb=None, str_qa=None,k):
        sent_emb = self.get_embeddings_for_data(sentence)
        #data_emb = self.get_embeddings_for_data(transfer_questions)
        top_questions = []

        #print("new_emb", sent_emb.shape, data_emb.shape)
        text_sims = cosine_similarity(data_emb,[sent_emb]).tolist()
        results_sims = zip(range(len(text_sims)), text_sims)
        sorted_similarities = sorted(results_sims, key=lambda x: x[1], reverse=True)

        #print("text_sims",sorted_similarities[:2])
        for idx, item in sorted_similarities[:k]:
                #if item[0] > 0.45:
                    top_questions.append(transfer_data[idx])

        #text_sims = cosine_similarity(strategy_emb,[sent_emb]).tolist()
        #results_sims = zip(range(len(text_sims)), text_sims)
        #sorted_similarities = sorted(results_sims, key=lambda x: x[1], reverse=True)
        #print("text_sims",sorted_similarities[:2])
        #for idx, item in sorted_similarities:
        #        top_questions.append(str_qa[idx])
        return top_questions

In [5]:
knnSearch = KnnSearch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
train_questions = [item["question"] for item in train_data]
train_questions_embedded = knnSearch.get_embeddings_for_data(train_questions)

In [7]:
test_questions = [item["question"] for item in test_data]
test_answers = [item["answers"] for item in test_data]

In [8]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is enabled
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [9]:
from transformers import pipeline

# Load the FlanT5-large model
pl = pipeline("text2text-generation", model="google/flan-t5-large", device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [19]:
def generate_prompt(few_shot_examples, question):
  result = ""
  for few_shot_example in few_shot_examples:
    result += f"Question: {few_shot_example['question']}\n"
    result += f"Answer: {'; '.join(few_shot_example['answers'])}\n\n"

  result += f"Question: {question}\n"
  result += "Answer: "
  return result

def postprocessing(answers):
  return 0

def generate_json(questions, generated_answers, ground_truths):
  data = []
  for question, generated, ground_truth in zip(questions, generated_answers, ground_truths):
      entry = {
          "question": question,
          "generated_answer": generated.split("; "),
          "ground_truth": ground_truth
      }
      data.append(entry)

    # Write the list of dictionaries to a JSON file
  with open("output_large_k_10.json", "w") as json_file:
      json.dump(data, json_file, indent=4)

  print("JSON file created successfully!")

In [20]:
def test_run(questions, ground_truths, k):
  all_prompts = []
  for question in questions:
    most_similar_questions = knnSearch.get_top_n_neighbours(sentence=question,
                                        data_emb=train_questions_embedded,
                                        transfer_data=train_data,
                                        k=k)

    prompt = generate_prompt(most_similar_questions, question)
    all_prompts.append(prompt)

  outputs = []
  batch_size = 10
  for i in range(0, len(all_prompts), batch_size):
    print(i)
    batch_prompts = all_prompts[i:i + batch_size]
    batch_outputs = pl(batch_prompts, max_new_tokens=50)
    outputs.extend(batch_outputs)
  generated_answers = [entry['generated_text'] for entry in outputs]

  #print(generated_answers)
  #print(len(generated_answers))
  generate_json(questions, generated_answers, ground_truths)

In [ ]:
test_run(test_questions, test_answers, 10)

0


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
